In [38]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
url_list = pd.read_csv('./data/series_urls.csv')

In [ ]:
base_url = 'https://en.wikipedia.org/'

# Go through the list of URLs for each series.
for index, row in url_list.iterrows():
    series = row['Series']
    # print(series)
    soup = BeautifulSoup(urlopen(row['URL']))
    cast_table = soup.find('table', class_='wikitable')
    rows = cast_table.find_all('tr')

    # Loop through each row in the table that contains housewife data.
    for row in rows[2:]:
        # Ignore Friends of the Housewives since they're not the main cast.
        if row.text.strip().lower() == 'friends of the housewives':
            break

        # Get the table data that has the housewife's name.
        td = row.find('td')

        # Get the full name text and split it into two parts, then replace any apostrophes with double
        # apostrophes to avoid SQL syntax errors.
        name = td.text.strip()
        name_parts = name.split(' ', 1)
        first_name = name_parts[0].replace("'", "''")
        last_name = name_parts[1].replace("'", "''")

        birthdate = None # Reset the birthdate each loop, otherwise wrong birthdate may be carried over.

        # Find the birthday of each housewife by checking if there's a hyperlink attached to their name.
        housewife_url = td.find('a')
        if housewife_url != None:
            housewife_url = urljoin(base_url, housewife_url.get("href"))
            req = requests.get(housewife_url)
            html = BeautifulSoup(req.text, 'html.parser')
            biography_table = html.find('table', class_='infobox')
            birthdate = biography_table.find('span', class_='bday')

            # Replace birthdate with text found on page if there is one.
            if birthdate != None:
                birthdate = birthdate.text
            
        print(f"('{first_name}', '{last_name}', '{birthdate}', null),")